In [262]:
import csv 
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d

In [263]:
# # owner__title__cpu/memory__extraName__sandbox

# janitha__CPU_Usage__cpu__centralized_plugin__m1
# janitha__CPU_Usage__cpu__centralized_plugin__m2
# janitha__CPU_Usage__cpu__centralized_plugin__m3
# janitha__Memory_Usage__memory__centralized_plugin__m1
# janitha__Memory_Usage__memory__centralized_plugin__m2
# janitha__Memory_Usage__memory__centralized_plugin__m3
# janitha__CPU_Usage__cpu__distributed_plugin__m1
# janitha__CPU_Usage__cpu__distributed_plugin__m2
# janitha__CPU_Usage__cpu__distributed_plugin__m3
# janitha__Memory_Usage__memory__distributed_plugin__m1
# janitha__Memory_Usage__memory__distributed_plugin__m2
# janitha__Memory_Usage__memory__distributed_plugin__m3

name_mapper = {
    "x_label" : "Time",
    "cpu" : "CPU (%)",
    "memory": "Memory (MB)",
    "m1" : "Machine 1",
    "m2" : "Machine 2",
    "m3" : "Machine 3",
}

# Create CSV Files

In [264]:
def convert_to_mib(memory):
    # Convert memory to MiB if necessary
    if 'GiB' in memory:
        memory_in_gib = float(memory[:-3])
        memory_in_mib = memory_in_gib * 1024
        return f"{memory_in_mib:.2f}MiB"
    elif 'MiB' in memory:
        return memory
    else:
        raise ValueError("Invalid memory format")


def extract_info(file_name):
    cpu_percentages = []
    mem_usages = []

    with open(file_name, 'r') as file:
        for line in file:
            values = line.split()
            
            cpu_percentages.append(float(values[2][:-1]))
            mem_usages.append(float(convert_to_mib(values[3])[:-3]))

    return cpu_percentages, mem_usages


def write_to_csv(cpu_percentages, mem_usages, output_file):
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([name_mapper["cpu"], name_mapper["memory"]])
        for i in range(len(cpu_percentages)):
            writer.writerow([cpu_percentages[i], mem_usages[i]])

In [265]:
def list_txt_files_in_folder(folder_path):
    txt_files = []
    try:
        for file in os.listdir(folder_path):
            if file.endswith(".txt") and os.path.isfile(os.path.join(folder_path, file)):
                txt_files.append(file)
    except Exception as e:
        print(f"Error occurred while listing .txt files: {e}")
    return txt_files

In [266]:
input_folder_path = "./txt/"
output_folder_path = "./output/"

files_in_folder = list_txt_files_in_folder(input_folder_path)

for file_name in files_in_folder:
    print(file_name)
    file_path = input_folder_path + file_name
    output_file_path = "./output/" + file_name.split(".")[0] + ".csv"

    cpu_percentages, mem_usages = extract_info(file_path)
    write_to_csv(cpu_percentages, mem_usages, output_file_path)
    
    print(f'Data has been written to {output_file_path}')

janitha__CPU_Usage__cpu__distributed_plugin__m1.txt
Data has been written to ./output/janitha__CPU_Usage__cpu__distributed_plugin__m1.csv
janitha__CPU_Usage__cpu__centralized_plugin__m1.txt
Data has been written to ./output/janitha__CPU_Usage__cpu__centralized_plugin__m1.csv
janitha__CPU_Usage__cpu__distributed_plugin__m2.txt
Data has been written to ./output/janitha__CPU_Usage__cpu__distributed_plugin__m2.csv
janitha__CPU_Usage__cpu__centralized_plugin__m2.txt
Data has been written to ./output/janitha__CPU_Usage__cpu__centralized_plugin__m2.csv
janitha__CPU_Usage__cpu__centralized_plugin__m3.txt
Data has been written to ./output/janitha__CPU_Usage__cpu__centralized_plugin__m3.csv
janitha__CPU_Usage__cpu__distributed_plugin__m3.txt
Data has been written to ./output/janitha__CPU_Usage__cpu__distributed_plugin__m3.csv
janitha__Memory_Usage__memory__centralized_plugin__m2.txt
Data has been written to ./output/janitha__Memory_Usage__memory__centralized_plugin__m2.csv
janitha__Memory_Usage_

# Generate Charts

In [267]:
def list_csv_files_in_folder(folder_path):
    csv_files = []
    try:
        for file in os.listdir(folder_path):
            if file.endswith(".csv") and os.path.isfile(os.path.join(folder_path, file)):
                csv_files.append(file)
    except Exception as e:
        print(f"Error occurred while listing .csv files: {e}")
    return csv_files

In [268]:
csv_files_in_folder = list_csv_files_in_folder(output_folder_path)
csv_files_in_folder_sorted = sorted(csv_files_in_folder)

result = {}

for item in csv_files_in_folder_sorted:
    parts = item.split("__")
    key = "__".join(parts[:-1])  # Join all parts except the last one
    value = parts[-1].split(".")[0]
    result.setdefault(key, []).append(value)

print(result)


{'janitha__CPU_Usage__cpu__centralized_plugin': ['m1', 'm2', 'm3'], 'janitha__CPU_Usage__cpu__distributed_plugin': ['m1', 'm2', 'm3'], 'janitha__Memory_Usage__memory__centralized_plugin': ['m1', 'm2', 'm3'], 'janitha__Memory_Usage__memory__distributed_plugin': ['m1', 'm2', 'm3']}


In [269]:
csv_files_in_folder = list_csv_files_in_folder(output_folder_path)
chart_folder_path = "./chart/"

for key, values in result.items():
    for value in values:
        file_path = output_folder_path + key + "__" + value + ".csv"
        usage_type = key.split("__")[2]
        title = " ".join(key.split("__")[1].split("_"))
        chart_folder = key.split("__")[0]
        usage_values = []
        
        with open(file_path, "r") as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                usage_value = float(row[name_mapper[usage_type]])
                usage_values.append(usage_value)
    
        x = np.arange(len(usage_values))

        y = np.array(usage_values)
        f = interp1d(x, y, kind='cubic')
        x_new = np.linspace(x.min(), x.max(), 300)
        y_smooth = f(x_new)
        
        plt.plot(x_new, y_smooth, marker='', linestyle='-')
        plt.title(title)
        plt.xlabel(name_mapper["x_label"])
        plt.ylabel(name_mapper[usage_type])
        plt.grid(True)
        plt.tight_layout()

        chart_file_name = key + "__" + value + ".png"
        chart_file_path = chart_folder_path + chart_file_name
        plt.savefig(chart_file_path, dpi=300)
        plt.clf()

<Figure size 640x480 with 0 Axes>

In [270]:
for key, values in result.items():
    for value in values:
        file_path = output_folder_path + key + "__" + value + ".csv"
        usage_type = key.split("__")[2]
        title = " ".join(key.split("__")[1].split("_"))
        chart_folder = key.split("__")[0]
        usage_values = []
        
        with open(file_path, "r") as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                usage_value = float(row[name_mapper[usage_type]])
                usage_values.append(usage_value)
    
        x = np.arange(len(usage_values))

        y = np.array(usage_values)
        f = interp1d(x, y, kind='cubic')
        x_new = np.linspace(x.min(), x.max(), 300)
        y_smooth = f(x_new)
        
        plt.plot(x_new, y_smooth, marker='', linestyle='-')
        plt.title(title)
        plt.xlabel(name_mapper["x_label"])
        plt.ylabel(name_mapper[usage_type])
        plt.grid(True)
        plt.tight_layout()

        chart_file_name = key + ".png"
        chart_file_path = chart_folder_path + chart_folder + "/" + chart_file_name
        plt.savefig(chart_file_path, dpi=300)
    plt.clf()

<Figure size 640x480 with 0 Axes>